In [1]:
import pandas as pd
import numpy as np

Đọc dữ liệu

In [2]:
patient = pd.read_csv("data/patient_info.csv")
visit = pd.read_csv("data/visit_records.csv")
billing = pd.read_csv("data/billing_info.csv")

lấy thông tin

In [3]:
patient.head()
patient.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40 entries, 0 to 39
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   patient_id  40 non-null     object
 1   full_name   40 non-null     object
 2   age         40 non-null     object
 3   gender      40 non-null     object
dtypes: object(4)
memory usage: 1.4+ KB


Làm sạch vấn đề thừa khoảng trắng , viết thường

In [4]:
patient["patient_id"] = (
    patient["patient_id"]
    .astype(str)
    .str.strip()
    .str.upper()
)
patient.head()

,patient_id,full_name,age,gender
0,BN100,LE VAN c,30,nu
1,BN101,HOANG van e,35t,M
2,BN102,pham thi d,50,m
3,BN103,LE VAN c,25,nu
4,BN104,Nguyen van A,30,nu


Chuẩn hóa full_name
vấn đề thừa space viết lung tung

In [5]:
patient["full_name"] = (
    patient["full_name"]
    .astype(str)
    .str.strip()
    .str.replace(r"\s+", " ", regex=True)
    .str.title()
)

patient.head()

,patient_id,full_name,age,gender
0,BN100,Le Van C,30,nu
1,BN101,Hoang Van E,35t,M
2,BN102,Pham Thi D,50,m
3,BN103,Le Van C,25,nu
4,BN104,Nguyen Van A,30,nu


xử lý age 
vấn đề có số âm
có text

In [6]:
patient["age"] = (
    patient["age"]
    .astype(str)
    .str.extract(r"(\d+)")
)

patient["age"] = pd.to_numeric(
    patient["age"],
    errors="coerce"
)

patient.loc[patient["age"] < 0, "age"] = np.nan

patient[["age"]].head()

,age
0,30.0
1,35.0
2,50.0
3,25.0
4,30.0


Chuẩn hóa gender
vấn đề m, female,nam,nu

In [7]:
def clean_gender(x):
    if pd.isna(x):
        return np.nan
    x = str(x).strip().lower()
    if x in ["m", "male", "nam"]:
        return "M"
    if x in ["f", "female", "nu", "nữ"]:
        return "F"
    return np.nan

patient["gender"] = patient["gender"].apply(clean_gender)

patient.head()

,patient_id,full_name,age,gender
0,BN100,Le Van C,30.0,F
1,BN101,Hoang Van E,35.0,M
2,BN102,Pham Thi D,50.0,M
3,BN103,Le Van C,25.0,F
4,BN104,Nguyen Van A,30.0,F


In [8]:
patient.to_csv("data/patient_info_clean.csv", index=False)

Đọc dữ liệu

In [9]:
visit.head()
visit.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 120 entries, 0 to 119
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   visit_id    120 non-null    object
 1   patient_id  120 non-null    object
 2   visit_date  120 non-null    object
 3   department  120 non-null    object
 4   diagnosis   99 non-null     object
dtypes: object(5)
memory usage: 4.8+ KB


Làm sạch

In [10]:
visit["patient_id"] = (
    visit["patient_id"]
    .str.strip()     # bỏ khoảng trắng đầu/cuối
    .str.upper()     # chuyển sang chữ HOA
)

In [11]:
visit["visit_date"] = pd.to_datetime(
    visit["visit_date"],
    format="mixed",
    dayfirst=True,
    errors="coerce"
)

In [12]:
visit["visit_date"].dtype

dtype('<M8[ns]')

Chuẩn hóa department

In [13]:
# Chuẩn hóa department
visit["department"] = (
    visit["department"]
    .str.strip()          # bỏ khoảng trắng
    .str.lower()          # về chữ thường
)

# Map về tên chuẩn
dept_map = {
    "noi": "Noi",
    "no i": "Noi",
    "ngoai": "Ngoai",
    "san": "San",
    "nhi": "Nhi"
}

visit["department"] = visit["department"].replace(dept_map)

# Kiểm tra kết quả
visit["department"].value_counts()


department
Nhi      34
San      32
Noi      29
Ngoai    25
Name: count, dtype: int64

Xử lý diagnosis

In [14]:
# Bỏ khoảng trắng + chữ thường
visit["diagnosis"] = visit["diagnosis"].str.strip().str.lower()

# Xóa ký tự rác
visit["diagnosis"] = visit["diagnosis"].replace(
    {"!@#$": None, "": None}
)

# Điền giá trị thiếu
visit["diagnosis"] = visit["diagnosis"].fillna("chua ro")

# Viết hoa chữ cái đầu
visit["diagnosis"] = visit["diagnosis"].str.title()

# Kiểm tra
visit["diagnosis"].value_counts()


diagnosis
Chua Ro      64
Dau Bung     20
Tieu Chay    19
Viem Phoi    17
Name: count, dtype: int64

In [15]:
visit.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 120 entries, 0 to 119
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   visit_id    120 non-null    object        
 1   patient_id  120 non-null    object        
 2   visit_date  120 non-null    datetime64[ns]
 3   department  120 non-null    object        
 4   diagnosis   120 non-null    object        
dtypes: datetime64[ns](1), object(4)
memory usage: 4.8+ KB


In [16]:
visit.to_csv("data/visit_records_clean.csv", index=False)


Đọc dữ liệu

In [17]:
billing.head()
billing.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 120 entries, 0 to 119
Data columns (total 3 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   visit_id            120 non-null    object
 1   total_fee           105 non-null    object
 2   insurance_coverage  120 non-null    object
dtypes: object(3)
memory usage: 2.9+ KB


Làm sạch blling_info total_fee

In [18]:
billing["visit_id"] = (
    billing["visit_id"]
    .astype(str)
    .str.strip()
    .str.upper()
)

In [19]:
def clean_fee(x):
    if pd.isna(x):
        return np.nan
    x = str(x).lower().strip()
    x = x.replace("đ", "").replace(".", "").replace(",", "")
    if "k" in x:
        return float(x.replace("k", "")) * 1000
    return float(x)

billing["total_fee"] = billing["total_fee"].apply(clean_fee)
billing

,visit_id,total_fee,insurance_coverage
0,VK2087,200000.0,0.8
1,VK2008,350000.0,0.8
2,VK2107,200000.0,0.8
3,VK2048,NaN,80%
4,VK2107,350000.0,80%
...,...,...,...
115,VK2022,300000.0,0.5
116,VK2024,1200000.0,50%
117,VK2107,200000.0,50%
118,VK2019,200000.0,-1


In [20]:
def clean_insurance(x):
    if pd.isna(x):
        return np.nan
    x = str(x).strip()
    if "%" in x:
        return float(x.replace("%", "")) / 100
    return float(x)

billing["insurance_coverage"] = billing["insurance_coverage"].apply(clean_insurance)

billing = billing[
    (billing["insurance_coverage"] > 0) &
    (billing["insurance_coverage"] <= 1)
]
billing


,visit_id,total_fee,insurance_coverage
0,VK2087,200000.0,0.8
1,VK2008,350000.0,0.8
2,VK2107,200000.0,0.8
3,VK2048,NaN,0.8
4,VK2107,350000.0,0.8
...,...,...,...
113,VK2051,300000.0,0.5
115,VK2022,300000.0,0.5
116,VK2024,1200000.0,0.5
117,VK2107,200000.0,0.5


In [21]:
billing = billing.dropna(
    subset=["total_fee", "insurance_coverage"],
    how="all"
)


In [22]:
billing = billing.sort_values(
    by=["visit_id", "insurance_coverage", "total_fee"],
    ascending=[True, False, False]
)

billing = billing.drop_duplicates(
    subset="visit_id",
    keep="first"
).reset_index(drop=True)

billing


,visit_id,total_fee,insurance_coverage
0,VK2000,NaN,0.8
1,VK2001,1200000.0,0.8
2,VK2004,200000.0,0.5
3,VK2008,350000.0,0.8
4,VK2010,300000.0,0.8
...,...,...,...
65,VK2110,1200000.0,0.8
66,VK2114,350000.0,0.8
67,VK2115,1200000.0,0.5
68,VK2116,200000.0,0.5


In [23]:
billing = billing.sort_values("visit_id").reset_index(drop=True)
billing


,visit_id,total_fee,insurance_coverage
0,VK2000,NaN,0.8
1,VK2001,1200000.0,0.8
2,VK2004,200000.0,0.5
3,VK2008,350000.0,0.8
4,VK2010,300000.0,0.8
...,...,...,...
65,VK2110,1200000.0,0.8
66,VK2114,350000.0,0.8
67,VK2115,1200000.0,0.5
68,VK2116,200000.0,0.5


In [24]:
billing.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 70 entries, 0 to 69
Data columns (total 3 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   visit_id            70 non-null     object 
 1   total_fee           63 non-null     float64
 2   insurance_coverage  70 non-null     float64
dtypes: float64(2), object(1)
memory usage: 1.8+ KB


In [25]:
billing.to_csv("data/billing_info_clean.csv", index=False)